# Peak finding

## Data for this notebook 

We will be manipulating Hubble eXtreme Deep Field (XDF) data, which was collected using the Advanced Camera for Surveys (ACS) on Hubble between 2002 and 2012. The image we use here is the result of 1.8 million seconds (500 hours!) of exposure time, and includes some of the faintest and most distant galaxies that had ever been observed. 

*The methods demonstrated here are also available in the [`photutils.detection` documentation](http://photutils.readthedocs.io/en/stable/detection.html).*

*The original authors of this notebook were Lauren Chambers, Erik Tollerud and Tom Wilson.*


In [ ]:
from astropy.io import fits
import astropy.units as u
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.visualization import ImageNormalize, LogStretch

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator

import numpy as np

from photutils.background import Background2D, MeanBackground
from photutils.centroids import centroid_2dg
from photutils.detection import find_peaks, DAOStarFinder, IRAFStarFinder

# Show plots in the notebook
%matplotlib inline

In [ ]:
xdf_image = CCDData.read('hlsp_xdf_hst_acswfc-60mas_hudf_f435w_v1_sci.fits')
# Define the mask
mask = xdf_image.data == 0
xdf_image.mask = mask

In [ ]:
mean, median, std = sigma_clipped_stats(xdf_image.data, sigma=3.0, maxiters=5, mask=xdf_image.mask)

In [ ]:
plt.style.use('../photutils_notebook_style.mplstyle')

## Source Detection with `find_peaks`

For more general source detection cases that do not require comparison with models, `photutils` offers the `find_peaks` function. 

This function simply finds sources by identifying local maxima above a given threshold and separated by a given distance, rather than trying to fit data to a given model. Unlike the previous detection algorithms, `find_peaks` does not necessarily calculate objects' centroids. Unless the `centroid_func` argument is passed a function like `photutils.centroids.centroid_2dg` that can handle source position centroiding, `find_peaks` will return just the integer value of the peak pixel for each source.

This algorithm is particularly useful for identifying non-stellar sources or heavily distorted sources in image data.

Let's see how it does:

In [ ]:
sources_findpeaks = find_peaks(xdf_image.data, mask=xdf_image.mask,
                               threshold=20. * std, box_size=29,
                               centroid_func=centroid_2dg)
print(sources_findpeaks)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
# Set up the normalization and colormap
norm_image = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch(), clip=False)
cmap = plt.get_cmap('viridis')
cmap.set_bad('white') # Show masked data as white
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image.data),
                      norm=norm_image, cmap=cmap)
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=30, marker='o',
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator())

def format_colorbar(bar):
    # Add minor tickmarks
    bar.ax.yaxis.set_minor_locator(LogLocator(subs=range(1, 10)))

    # Force the labels to be displayed as powers of ten and only at exact powers of ten
    bar.ax.set_yticks([1e-4, 1e-3, 1e-2])
    labels = [f'$10^{{{pow:.0f}}}$' for pow in np.log10(bar.ax.get_yticks())]
    bar.ax.set_yticklabels(labels)

format_colorbar(cbar)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')),
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('find_peaks Sources')

And a closer look:

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_findpeaks)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['y_peak'] - cutout_size), int(src['y_peak'] + cutout_size)),
           slice(int(src['x_peak'] - cutout_size), int(src['x_peak'] + cutout_size)))
    ax.imshow(xdf_image[slc], norm=norm_image)
    src_id = np.where((sources_findpeaks['x_peak'] == src['x_peak']) &
                      (sources_findpeaks['y_peak'] == src['y_peak']))[0][0]
    ax.text(2, 2, str(src_id), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

## Comparing Detection Methods

Let's compare how each of these different strategies did.

First, we repeat our earlier detection using [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) and [`IRAFStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.IRAFStarFinder.html#photutils.detection.IRAFStarFinder).


In [ ]:
daofind = DAOStarFinder(fwhm=5.0, threshold=20. * std)
sources_dao = daofind(np.ma.masked_where(xdf_image.mask, xdf_image))

iraffind_match = IRAFStarFinder(fwhm=5.0, threshold=20. * std,
                                sharplo=0.2, sharphi=1.0,
                                roundlo=-1.0, roundhi=1.0,
                                minsep_fwhm=0.0)
sources_iraf_match = iraffind_match(np.ma.masked_where(xdf_image.mask, xdf_image))

In [ ]:
print('''DAOStarFinder: {} sources
IRAFStarFinder: {} sources
find_peaks: {} sources'''.format(len(sources_dao), len(sources_iraf_match), len(sources_findpeaks)))

Next, how many of these sources match? We can answer this question by using [sets](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset) to compare the centroids of the different sources (rounding to the nearest integer).

In [ ]:
# Make lists of centroid coordinates
centroids_dao = [(x, y) for x, y in sources_dao['xcentroid', 'ycentroid']]
centroids_iraf = [(x, y) for x, y in sources_iraf_match['xcentroid', 'ycentroid']]
centroids_findpeaks = [(x, y) for x, y in sources_findpeaks['x_centroid', 'y_centroid']]

# Round those coordinates to the ones place and convert them to be sets
rounded_centroids_dao = set([(round(x, 0), round(y, 0)) for x, y in centroids_dao])
rounded_centroids_iraf = set([(round(x, 0), round(y, 0)) for x, y in centroids_iraf])
rounded_centroids_findpeaks = set([(round(x, 0), round(y, 0)) for x, y in centroids_findpeaks])

# Examine the intersections of different sets to determine which sources are shared
all_match = rounded_centroids_dao.intersection(rounded_centroids_iraf).intersection(rounded_centroids_findpeaks)
dao_iraf_match = rounded_centroids_dao.intersection(rounded_centroids_iraf)
dao_findpeaks_match = rounded_centroids_dao.intersection(rounded_centroids_findpeaks)
iraf_findpeaks_match = rounded_centroids_iraf.intersection(rounded_centroids_findpeaks)

print('''Matching sources found by:
    All methods: {}
    DAOStarFinder & IRAFStarFinder: {}
    DAOStarFinder & find_peaks: {}
    IRAFStarFinder & find_peaks: {}'''
      .format(len(all_match), len(dao_iraf_match),
              len(dao_findpeaks_match), len(iraf_findpeaks_match)))

And just for fun, let's plot these matching sources. (The colors chosen to represent different sets are from [Paul Tol's guide for accessible color schemes](https://personal.sron.nl/~pault/).)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image), norm=norm_image)
ax1.scatter([x for x, y in list(dao_findpeaks_match)], [y for x, y in list(dao_findpeaks_match)],
            s=30, marker='s', lw=1,  facecolor='None', edgecolor='#EE7733',
            label='Found by DAO and find_peaks')
ax1.scatter([x for x, y in list(dao_iraf_match)], [y for x, y in list(dao_iraf_match)],
            s=30, marker='D', lw=1, facecolor='None', edgecolor='#EE3377',
            label='Found by DAO and IRAF')
ax1.scatter([x for x, y in list(iraf_findpeaks_match)], [y for x, y in list(iraf_findpeaks_match)],
            s=30, marker='o', lw=1, facecolor='None', edgecolor='#0077BB',
            label='Found by IRAF and find_peaks')
ax1.scatter([x for x, y in list(all_match)], [y for x, y in list(all_match)],
            s=30, marker='o', lw=1.2, linestyle=':',facecolor='None', edgecolor='#BBBBBB',
            label='Found by all methods')

# Add legend
ax1.legend(ncol=2, loc='lower center', bbox_to_anchor=(0.5, -0.25))

# Define labels
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Sources Found by Different Methods')

*Remember that you can double-click on the plot to zoom in and look around!*

## Custom Detection Algorithms

If none of the algorithms we've reviewed above do exactly what you need, `photutils` also provides infrastructure for you to generate and use your own source detection algorithm: the `StarFinderBase` object can be inherited and used to develop new star-finding classes. Take a look at the [documentation](https://photutils.readthedocs.io/en/latest/api/photutils.detection.StarFinderBase.html#photutils.detection.StarFinderBase) for more information.

If you do go that route, remember that `photutils` is open-developed; you would be very welcome to [open a pull request](https://github.com/astropy/photutils/blob/master/CONTRIBUTING.rst) and incorporate your new star finder into the `photutils` source code &ndash; for everyone to use!